In [8]:
"""Test script that runs all the vehicles in both Excel and Python FASTSim for both UDDS and HWFET cycles."""

import xlwings as xw
import pandas as pd
import time
import numpy as np
import os
import sys
import importlib
import xlwings as xw
from math import isclose
import importlib
import pickle

# local modules
from fastsim import simdrive, vehicle, cycle, simdrivelabel
importlib.reload(simdrivelabel)

<module 'fastsim.simdrivelabel' from 'c:\\users\\cbaker2\\documents\\projects\\fastsim\\github\\fastsim\\fastsim\\simdrivelabel.py'>

In [3]:
def run_python_fastsim(verbose=True):
    """Runs python fastsim through 26 vehicles and returns list of dictionaries 
    containing scenario descriptions.
    
    Arguments:
    **********
    verbose : Boolean
        if True, print progress"""

    t0 = time.time()

    vehicles = np.arange(1, 27)

    print('Instantiating classes.')
    print()

    res_python = {}

    for vehno in vehicles:
        if verbose:
            print('vehno =', vehno)
        veh_jit = vehicle.Vehicle(vehno).get_numba_veh()
        res_python[veh_jit.Scenario_name] = simdrivelabel.get_label_fe(veh_jit)
                                   
    t1 = time.time()
    print()
    print('Elapsed time: ', round(t1 - t0, 2), 's')

    return res_python

In [9]:
def run_excel_fastsim():
    """Runs excel fastsim through 26 vehicles and returns list of dictionaries 
    containing scenario descriptions."""

    t0 = time.time()

    # initial setup
    wb = xw.Book('FASTSim.xlsm')  # FASTSim.xlsm must be open
    # capture sheets with results of interest
    sht_veh = wb.sheets('VehicleIO')
    sht_vehnames = wb.sheets('SavedVehs')
    # setup macros to run from python
    app = wb.app
    load_veh_macro = app.macro("FASTSim.xlsm!reloadVehInfo")
    run_macro = app.macro("FASTSim.xlsm!run.run")

    vehicles = np.arange(1, 28)
    res_excel = {}
    
    for vehno in vehicles:
        print('vehno =', vehno)
        # running a particular vehicle and getting the result
        res_dict = {}
        # set excel to run vehno
        sht_veh.range('C6').value = vehno
        load_veh_macro()
        run_macro()
        # lab results (unadjusted)
        res_dict['labUddsMpgge'] = sht_veh.range('labUddsMpgge').value
        res_dict['labHwyMpgge'] = sht_veh.range('labHwyMpgge').value
        res_dict['labCombMpgge'] = sht_veh.range('labCombMpgge').value
        res_dict['labUddsKwhPerMile'] = sht_veh.range('labUddsKwhPerMile').value
        res_dict['labHwyKwhPerMile'] = sht_veh.range('labHwyKwhPerMile').value
        res_dict['labCombKwhPerMile'] = sht_veh.range('labCombKwhPerMile').value

        # adjusted results
        res_dict['adjUddsMpgge'] = sht_veh.range('adjUddsMpgge').value
        res_dict['adjHwyMpgge'] = sht_veh.range('adjHwyMpgge').value
        res_dict['adjCombMpgge'] = sht_veh.range('adjCombMpgge').value
        res_dict['adjUddsKwhPerMile'] = sht_veh.range('adjUddsKwhPerMile').value
        res_dict['adjHwyKwhPerMile'] = sht_veh.range('adjHwyKwhPerMile').value
        res_dict['adjCombKwhPerMile'] = sht_veh.range('adjCombKwhPerMile').value

        for key in res_dict.keys():
            if (res_dict[key] == '') | (res_dict[key] == None):
                res_dict[key] = 0
        
        res_excel[sht_vehnames.range('B' + str(vehno + 2)).value] = res_dict

    t1 = time.time()
    print()
    print('Elapsed time: ', round(t1 - t0, 2), 's')

    return res_excel

In [5]:
def compare(res_python, res_excel):
    """Finds common vehicle names in both excel and python 
    (hypothetically all of them, but there may be discrepancies) and then compares
    fuel economy results.  
    Arguments: results from run_python_fastsim and run_excel_fastsim
    Returns dict of comparsion results."""

    common_names = set(res_python.keys()) & set(res_excel.keys())

    ERR_TOL = 0.02

    res_keys = ['labUddsMpgge', 'labHwyMpgge', 'labCombMpgge', 
        'labUddsKwhPerMile', 'labHwyKwhPerMile', 'labCombKwhPerMile',
        'adjUddsMpgge', 'adjHwyMpgge', 'adjCombMpgge', 
        'adjUddsKwhPerMile', 'adjHwyKwhPerMile', 'adjCombKwhPerMile', ]

    res_comps = {}
    for vehname in common_names:
        print('\n')
        print(vehname)
        print('***'*7)
        res_comp = {}
        for res_key in res_keys:
            if not(isclose(res_python[vehname][res_key], 
                res_excel[vehname][res_key],
                rel_tol=ERR_TOL, abs_tol=ERR_TOL)):
                res_comp[res_key + '_frac_err'] = (
                    res_python[vehname][res_key] -
                    res_excel[vehname][res_key]) / res_excel[vehname][res_key]
            else:
                res_comp[res_key + '_frac_err'] = 0.0
            if res_comp[res_key + '_frac_err'] != 0.0:
                print(
                    res_key + ' error = {:.3g}%'.format(res_comp[res_key + '_frac_err'] * 100))
                
        if (np.array(list(res_comp.values())) == 0).all():
            print(f'All values within error tolerance of {ERR_TOL:.3g}')

        res_comps[vehname] = res_comp.copy()
    return res_comps

In [2]:
res_python = run_python_fastsim(verbose=False)

Instantiating classes.



TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [10]:
res_excel = run_excel_fastsim()
pickle.dump(res_excel, open('res_excel.p', 'wb'))
# res_excel = pickle.load(open('res_excel.p', 'rb'))

vehno = 1
vehno = 2
vehno = 3
vehno = 4
vehno = 5
vehno = 6
vehno = 7
vehno = 8
vehno = 9
vehno = 10
vehno = 11
vehno = 12
vehno = 13
vehno = 14
vehno = 15
vehno = 16
vehno = 17
vehno = 18
vehno = 19
vehno = 20
vehno = 21
vehno = 22
vehno = 23
vehno = 24
vehno = 25
vehno = 26
vehno = 27

Elapsed time:  163.56 s


In [ ]:
res_comps = compare(res_python, res_excel)

Instantiating classes.



In [ ]:
vehstr = "2016 AUDI A3 4cyl 2WD"
for key in res_excel[vehstr].keys():
    print(key + ' : {', end='')
    print('excel : {:.4g}, '.format(res_excel[vehstr][key]), end='')
    print('python:{:.4g}'.format(res_python[vehstr][key]) + '}')